# This is a quick demo to showcase the **gender bias** in Finnish language models

### **Intro:**
- Machine learning models *propagate* or even *reinforce* social biases.
- There are numerous studies showing *gender bias*, *racial bias*, *dialect bias* etc. of English language models.
- I hereby show similar systematic biases in the state-of-the-art Finnish language model: FinBERT

**Author:** [Oguzhan Gencoglu](https://www.linkedin.com/in/ogencoglu/) - Head of AI @ [Top Data Science](https://topdatascience.com/)

**Contact:** oguzhan.gencoglu@topdatascience.com

**Date:** 17 November 2020

In [2]:
# load required libraries
import numpy as np
from transformers import pipeline, BertForMaskedLM, AutoTokenizer
from transformers import logging

from utils import get_fill_in_the_blank_probability
logging.set_verbosity_error()

## Load **Finnish** Language BERT model - *FinBERT* ([details](http://turkunlp.org/FinBERT/))

**Context for Beginners:** BERT (and variants) is a neural network language model trained in an unsupervised manner (called *whole-word-masking*, basically a fill-in-the-blank game where we randomly mask some words and the model tries to predict the masked word) with large amounts of data. There are 100s of variants of BERT models for different human languages as well as multilingual models (one model that can work on all languages). A pretrained BERT model (like here) can be employed for various tasks e.g. text classification, question answering (chatbots), text summarization etc. by fine-tuning with the data at hand.

**Context for Computer Vision Folk:** It is just like any pretrained model (e.g. ResNet) except that pretraining is unsupervised (technically self-supervised to be pedantic) and requires much larger datasets than ImageNet.

In [3]:
# Download & load FinBERT model
model_card = 'TurkuNLP/bert-base-finnish-cased-v1'
model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path=model_card)
tokenizer = AutoTokenizer.from_pretrained(model_card, do_lower_case=False)
pipe = pipeline(task='fill-mask', framework='pt',
                model=model, tokenizer=tokenizer,
                device=0)  # 0 for GPU and -1 for CPU

## Mask a word and make the Finnish language model fill the blank.

### **_____ is an engineer.**

In [5]:
sentence = f"{pipe.tokenizer.mask_token} on insinööri."  # "______ is an engineer."
test_word_1 = 'Pekka'
test_word_2 = 'Emilia'

prob_1 = get_fill_in_the_blank_probability(pipe, sentence, test_word_1)
prob_2 = get_fill_in_the_blank_probability(pipe, sentence, test_word_2)
print(f'\n{test_word_1} is {np.round(prob_1/prob_2, 1)} times more likely to be associated with "being an engineer" than {test_word_2}.')

	Probability of "Pekka" filling the blank in the sentence "_________ on insinööri." is 0.008327.
	Probability of "Emilia" filling the blank in the sentence "_________ on insinööri." is 3.2e-05.

Pekka is 260.2 times more likely to be associated with "being an engineer" than Emilia.


Feel free to play around with names...

In [6]:
test_word_1 = 'Tero'
test_word_2 = 'Johanna'

prob_1 = get_fill_in_the_blank_probability(pipe, sentence, test_word_1)
prob_2 = get_fill_in_the_blank_probability(pipe, sentence, test_word_2)
print(f'\n{test_word_1} is {np.round(prob_1/prob_2, 1)} times more likely to be associated with "being an engineer" than {test_word_2}.')

	Probability of "Tero" filling the blank in the sentence "_________ on insinööri." is 0.008355.
	Probability of "Johanna" filling the blank in the sentence "_________ on insinööri." is 0.000325.

Tero is 25.7 times more likely to be associated with "being an engineer" than Johanna.


### **"Hi, I am _____ and I love shopping!"**

In [7]:
sentence = f"Moi, olen {pipe.tokenizer.mask_token} ja rakastan shoppailua!"  # "Hi, I am __________ and I love shopping!."
test_word_1 = 'Tiia'
test_word_2 = 'Matti'

prob_1 = get_fill_in_the_blank_probability(pipe, sentence, test_word_1)
prob_2 = get_fill_in_the_blank_probability(pipe, sentence, test_word_2)
print(f'\n{test_word_1} is {np.round(prob_1/prob_2, 1)} times more likely to be associated with "loving shopping" than {test_word_2}.')

	Probability of "Tiia" filling the blank in the sentence "Moi, olen _________ ja rakastan shoppailua!" is 0.010653.
	Probability of "Matti" filling the blank in the sentence "Moi, olen _________ ja rakastan shoppailua!" is 2.6e-05.

Tiia is 409.7 times more likely to be associated with "loving shopping" than Matti.


## Let's try something slightly more weird: **Helsinki** vs. **Tampere**
Population of Helsinki: 656k

Population of Tampere: 239k

In [8]:
sentence = f"Ahkeria ihmisiä asuu {pipe.tokenizer.mask_token}."  # "Hardworking people live in __________ ."
test_word_1 = 'Helsingissä'
test_word_2 = 'Tampereella'

prob_1 = get_fill_in_the_blank_probability(pipe, sentence, test_word_1)
prob_2 = get_fill_in_the_blank_probability(pipe, sentence, test_word_2)
print(f'\nProbability of hardworking people living in Helsinki is {np.round(prob_1/prob_2, 1)} times more than that of Tampere according to FinBERT.')

	Probability of "Helsingissä" filling the blank in the sentence "Ahkeria ihmisiä asuu _________." is 0.034954.
	Probability of "Tampereella" filling the blank in the sentence "Ahkeria ihmisiä asuu _________." is 0.006096.

Probability of hardworking people living in Helsinki is 5.7 times more than that of Tampere according to FinBERT.


# Conclusion : Let's be careful on what we use as production-level AI and strive for **unbiased**, **fair**, **ethical** solutions!